In [ ]:
import numpy as np
import pandas as pd
from chisq import EclipseFit
from etv_plots import nice_units, pretty_print, Plotter, ecl_time_to_etv, lsq_fit
import emcee
import corner
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('lines', linewidth=2)

In [ ]:
system = '5095'

In [ ]:
# see spherical triangle identities, e.g. in Appendix D of Borkovits (2015)
def im(i1, i2, Omega2):
    return np.arccos(np.cos(i1)*np.cos(i2) + np.sin(i1)*np.sin(i2)*np.cos(Omega2))

def n1(i1, i2, Omega2):
    n1 = np.arccos((np.cos(i2) - np.cos(i1)*np.cos(im(i1, i2, Omega2)))/(np.sin(i1)*np.sin(im(i1, i2, Omega2))))
    return np.where(np.sin(Omega2) < 0, n1, np.pi - n1)

def g1(i1, i2, Omega2, omega1):
    g1 = omega1 - n1(i1, i2, Omega2)
    return np.where(np.sin(Omega2) > 0, g1, g1 + np.pi)

def h(im, g1, e1):
    return np.cos(im)**2 - e1**2/2*np.sin(im)**2 * (3 - 5*np.cos(2*g1))

def k_sq(e1, h):
    return 5*e1**2/(1 - e1**2)*(1 - h)/(h + 4*e1**2)

print(np.degrees(g1(np.radians(85.571), np.radians(160.6), np.radians(77.2), np.radians(108.9))))
print(np.degrees(im(np.radians(85.571), np.radians(160.6), np.radians(77.2))))

# Best Fit

In [ ]:
chain = np.load('mcmc_out/' + system + '_chains_1.npy')
prob = np.load('mcmc_out/' + system + '_probs_1.npy')
ndim = chain.shape[-1]
good_walkers = -2*prob[:, -1] < 300
print('Good walkers: {}/{}'.format(np.sum(good_walkers), chain.shape[0]))
best_indx = prob.flatten()[np.nonzero(prob.flatten())].argmax()
print('Best chi-squared:', -2*prob.flatten()[best_indx])
x = chain.reshape(-1, ndim)[best_indx].copy()
print('\nBest Fit')
print(pretty_print(x))
y = np.median(chain.reshape(-1, ndim), axis=0)
print('\nMedian')
print(pretty_print(y))

In [ ]:
fit = EclipseFit(system)
ecl_model, rv_model = fit.get_residuals(x, safe=False)
dbdt, b0 = fit.impact_regression(ecl_model)
print(b0, dbdt)
#print(-2*fit.evaluate(x))
#plt.plot(ecl_model['A']['model_t'], b0['A'] + dbdt['A']*ecl_model['A']['model_t'])
#plt.plot(ecl_model['A']['model_t'], ecl_model['A']['model_b'])
#ecl_model['A']

In [ ]:
#import cProfile
#cProfile.run('fit.get_residuals(x)', sort='time')

# Data

In [ ]:
plt.rc('font', size=16)
plot = Plotter(system)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
T0, P = lsq_fit(ecl_model['A']['data_t'])
ax.errorbar(ecl_model['A']['data_t'], 86400*ecl_time_to_etv(ecl_model['A']['data_t'], P, T0), 
            86400*ecl_model['A']['data_err'], linestyle='None', color='k', marker='o')
ax.set_xlabel('Time (BJD-2454900)', fontsize=20)
ax.set_ylabel('O-C (seconds)', fontsize=20)
#plt.savefig('kic5095_etv_raw_unfolded.png', dpi='figure')

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
T0, P = lsq_fit(ecl_model['A']['data_t'])
ax.errorbar(np.remainder(ecl_model['A']['data_t'] - 95.923, 235.9)/235.9, 
            86400*ecl_time_to_etv(ecl_model['A']['data_t'], P, T0), 
            86400*ecl_model['A']['data_err'], linestyle='None', color='k', marker='o')
ax.set_xlabel('Outer Orbital Phase', fontsize=20)
ax.set_ylabel('O-C (seconds)', fontsize=20)
#plt.savefig('kic5095_etv_raw_folded.png', dpi='figure')

In [ ]:
if len(plot.ecl_stars) == 2:
    plot.etv_together(x)

# Model Fit

In [ ]:
plot.etv_residuals(x, 'A', phased=True)
#plt.savefig('etv_residuals_phased.png', dpi='figure')
plot.etv_residuals(x, 'A', phased=False)
#plt.savefig('etv_residuals_unphased.png', dpi='figure')
try:
    plot.etv_residuals(x, 'B', phased=True)
    plot.etv_residuals(x, 'B', phased=False)
except:
    pass

In [ ]:
plot.rv_residuals(x, phased=True)
plot.rv_residuals(x, phased=False)
#plt.savefig('rv_residuals.eps', dpi='figure')
plot.impact_fit_quality(x)

# TESS Eclipses

In [ ]:
plot_tess = Plotter(system)
tess_points = pd.DataFrame([[3789.25839, 0.00063], [3807.86757, 0.00067]], index=[203, 204], columns=['data_t', 'data_err'])
plot_tess.ecl_data['A'] = plot_tess.ecl_data['A'].append(tess_points)
plot_tess.etv_residuals(x, 'A', phased=True, ecl_max=4000)
plot_tess.etv_residuals(x, 'A', phased=False, ecl_max=4000)

# MCMC Diagnostics

In [ ]:
plt.plot(nice_units(chain)[...,0].T);
plt.xlabel('Generation')

In [ ]:
plt.plot(-2*prob.T)
plt.yscale('log')
plt.xlabel('Generation'); plt.ylabel('$\chi^2$')

In [ ]:
print(-2*prob[:,-1])

In [ ]:
dof = sum(map(len, plot.ecl_data.values())) + sum(map(len, plot.rv_data.values()))
print('Reduced chi^2: {:.4f}'.format(-2*prob.flatten()[best_indx]/dof))

In [ ]:
emcee.autocorr.integrated_time(chain[3,:,1])

In [ ]:
if system == '5095':
    labels = ['$P_1$', '$T_{01}$', '$i_1$', '$e_1$', '$\omega_1$',
              '$P_2$', '$T_{p2}$', '$e_2\cos\omega_2$', '$e_2\sin\omega_2$', '$i_2$', '$\Omega_2$',
              '$M_A$', '$M_B$', '$M_p$', '$\gamma_{NOT}$', '$\gamma_{CAHA}$']
corner.corner((nice_units(chain[:, 5000:, :].reshape(-1, 17)).T[chain.reshape(-1, 17).std(axis=0) > 1e-10]).T,
              labels=labels)
#plt.savefig('5095_corner.eps', dpi='figure')

# Long term evolution

In [ ]:
sim = plot.set_up_sim(x)
N = 1000
prec = np.zeros((N,6))
ts = np.linspace(0, 1600, N)
for i in range(N):
    sim.integrate(ts[i])
    prec[i,0] = ts[i]
    prec[i,1] = sim.particles[1].inc
    prec[i,2] = sim.particles[2].a
    prec[i,3] = sim.particles[2].e
    prec[i,4] = sim.particles[1].omega
#plt.plot(prec[...,0], np.degrees(prec[...,1]))
prec[...,5] = prec[...,2]*(1 - prec[...,3]**2)/(1 + prec[...,3]*np.sin(prec[...,4])) * np.cos(prec[...,1])/(1.2*0.00465)
plt.plot(prec[...,0], np.degrees(prec[...,4]))
#plt.ylabel('Eccentricity'); plt.xlabel('Time')

In [ ]:
sim = plot.set_up_sim(x)
N = 1000
prec = np.zeros((N,5))
ts = np.linspace(0, 300000, N)
for i in range(N):
    sim.integrate(ts[i])
    prec[i,0] = ts[i]
    prec[i,1] = sim.particles[1].inc
    prec[i,2] = sim.particles[2].inc
    prec[i,3] = sim.particles[2].Omega - sim.particles[1].Omega
    prec[i,4] = sim.particles[1].omega

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(prec[...,0]/365.25, np.degrees(im(*prec[...,1:4].T)), color='k')
ax1.plot(prec[...,0]/365.25, np.degrees(g1(*prec[...,1:5].T)), color='r')

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'), figsize=(16,9))
azimuths = np.radians(np.linspace(0, 360, 180))
zeniths = np.linspace(0,180,180)
r, theta = np.meshgrid(zeniths, azimuths)
values = h(np.radians(r), theta, x[3])
ax.contour(theta, r, values, 15)
ax.plot(np.linspace(0, 2*np.pi, 100), 90.0*np.ones(100), color='k', linestyle='--')
ax.set_yticklabels([])
ax.set_ylim(0, 180)
ax.grid(False)
mask = prob < 1000
#ax.scatter(g1(chain[...,2], chain[...,9], chain[...,10], chain[...,4])[mask].flatten(), 
#           np.degrees(im(chain[...,2], chain[...,9], chain[...,10]))[mask].flatten(), 
#           marker='.', color='k')
plt.plot(g1(*prec[...,1:5].T), np.degrees(im(*prec[...,1:4].T)), c='k')
#plt.plot(np.linspace(0, 2*np.pi, 100), np.array(100*[37.76]))
#plt.savefig('fl level curves.png', dpi='figure')

# Derived system quantities

In [ ]:
plt.hist(np.degrees(im(chain[:,5000:,2], chain[:,5000:,9], chain[:,5000:,10])).flatten(), 
         bins=20, density=True, color='0.2')
plt.xlabel('Mutual Inclination ($^\circ$)', fontsize=20)
#plt.savefig('im_constraint.png', dpi='figure', bbox_inches="tight")
i_m_95_upper = np.percentile(np.degrees(im(chain[:,5000:,2], chain[:,5000:,9], chain[:,5000:,10])).flatten(), 95)
print('95% upper limit of mutual inclination: {:.3f} deg'.format(i_m_95_upper))

In [ ]:
plt.hist(np.degrees(g1(chain[:,5000:,2], chain[:,5000:,9], chain[:,5000:,10], chain[:,5000:,4]).flatten()), 
         bins=20, density=True, color='0.2')
plt.xlabel('Dynamical Argument of Periastron')

In [ ]:
def stab_criterion(mu, e):
    return 1.6 + 5.1 * e - 2.22 * e**2 + 4.12 * mu - 4.27 * e * mu - 5.09 * mu**2 + 4.61 * e**2 * mu**2

def crit_radius(P1, P2, e, mA, mB):
    mu = mB/(mA + mB)
    return (P2/P1)**(2/3) / stab_criterion(mu, e)

a_c = crit_radius(chain[...,0], chain[...,5], chain[...,3], chain[...,11], chain[...,12])
plt.hist(a_c.flatten(), range=[1.45, 1.6], bins=30, color='k', density=True);
plt.xlabel(r'$a_{planet}/a_c$')
print('a_planet/a_c = {:.3f} ± {:.3f}'.format(a_c.mean(), a_c.std()))

In [ ]:
from scipy.special import ellipk, ellipkinc

def K(k_sq_):
    return np.where(k_sq_ < 1, ellipk(k_sq_), ellipkinc(np.arcsin(1/np.sqrt(k_sq_)), k_sq_))

# Farago & Laskar (2010) Eq. 2.32
def prec_timescale(P1, P2, e1, e2, im, g1, mA, mB):
    h_ = h(im, g1, e1)
    k_sq_ = k_sq(e1, h_)
    return 8/(3*np.pi) * (mA + mB)**2/(mA * mB) * (P2**7/P1**4)**(1/3) * K(k_sq_) * \
            (1 - e2**2)**2/np.sqrt((1 - e1**2) * (h_ + 4*e1**2))

T_precs = prec_timescale(chain[...,0], chain[...,5], chain[...,3], 
                         np.sqrt(chain[...,7]**2 + chain[...,8]**2), 
                         im(chain[...,2], chain[...,9], chain[...,10]), 
                         g1(chain[...,2], chain[...,9], chain[...,10], chain[...,4]), 
                         chain[...,11], chain[...,12])
plt.hist(T_precs[:,:5000].flatten()/365.25, bins=30, range=[T_precs.min()/365.25, 89], color='k', density=True)
plt.xlabel('Precession timescale (years)')
print('T_prec = {:.3f} ± {:.3f}'.format(T_precs.mean()/365.25, T_precs.std()/365.25))

In [ ]:
# minimum mutual inclination to be guaranteed to transit

R_A = 1.45*0.00465
R_B = 1.34*0.00465
G = 0.00029591220828559104

delta = R_A + R_B

a_bin = (G*(chain[...,11] + chain[...,12])*chain[...,0]/(4*np.pi**2))**(1/3)
a_p = (G*(chain[...,11] + chain[...,12])*chain[...,5]/(4*np.pi**2))**(1/3)

mu_A = chain[...,11]/(chain[...,11] + chain[...,12])
mu_B = chain[...,12]/(chain[...,11] + chain[...,12])

i_m_lim_A = delta*(1/a_bin - mu_A*1/a_p) - R_A/a_p
i_m_lim_B = delta*(1/a_bin - mu_B*1/a_p) - R_B/a_p

print('Minimum i_m to transit A: {:.2f} deg'.format(np.degrees(i_m_lim_A.mean())))
print('Minimum i_m to transit B: {:.2f} deg'.format(np.degrees(i_m_lim_B.mean())))

# probability of transit

P_A = np.where(im(chain[...,2], chain[...,9], chain[...,10]) > i_m_lim_A, 1, (im(chain[...,2], chain[...,9], chain[...,10]) + R_A/a_p)/(delta*(1/a_bin - mu_B/a_p)))
P_B = np.where(im(chain[...,2], chain[...,9], chain[...,10]) > i_m_lim_B, 1, (im(chain[...,2], chain[...,9], chain[...,10]) + R_B/a_p)/(delta*(1/a_bin - mu_A/a_p)))

print('Probability of transiting A: {:.2f}%'.format(100*P_A.mean()))
print('Probability of transiting B: {:.2f}%'.format(100*P_B.mean()))

In [ ]:
# Gaia
G = 2.959e-4
a_reflex = chain[...,13]/(chain[...,11] + chain[...,12])**(2/3) * (G/(4*np.pi**2) * chain[...,5]**2)**(1/3)
print('Reflex motion of binary: {:.5f} mas'.format(0.8216*a_reflex.mean()))

# Direct imaging
a_plan = (G*(chain[...,11] + chain[...,12])/(4*np.pi**2))**(1/3) * chain[...,5]**(2/3)
print('Separation of planet from binary: {:.4} as'.format(0.8216/1000*a_plan.mean()))

In [ ]:
plt.scatter(*(nice_units(chain[:, 3000:,:].reshape(-1, ndim))[...,11:13]).T, marker='.', s=1, alpha=0.01)
L_tot = 4.66
a = 3.5
plt.plot(np.linspace(1.0, 1.4, 100), (L_tot - np.linspace(1.0, 1.4, 100)**a)**(1/a), color='k')
plt.xlabel('$m_A$'); plt.ylabel('$m_B$');

In [ ]:
plt.hist(((chain[:, 3000:,:].reshape(-1, ndim)[...,11:13])**a).sum(axis=1), bins=30)
plt.xlabel('$L$ $(L_\odot)$')

In [ ]:
plt.hist((chain[:, 3000:,11]/chain[:,3000:,12]).flatten(), bins=30)
plt.xlabel('Binary mass ratio');

# Output for paper

In [ ]:
burn_in = 5000
flatchain = chain[:,burn_in:,:].reshape(-1, ndim)
paper_chain = np.empty_like(flatchain)
paper_chain[...,:2] = flatchain[...,:2] # P_1 and T_01
paper_chain[...,2] = flatchain[...,3] # e_1
paper_chain[...,3] = np.degrees(flatchain[...,2]) # i_1
paper_chain[...,4] = np.degrees(flatchain[...,4]) # omega_1
paper_chain[...,5:7] = flatchain[...,5:7] # P_2 and T_p2
paper_chain[...,7] = np.sqrt(flatchain[...,7]**2 + flatchain[...,8]**2) # e_2
paper_chain[...,8] = np.degrees(flatchain[...,9]) # i_2
paper_chain[...,9] = np.degrees(np.arctan2(flatchain[...,8], flatchain[...,7])) # omega_2
paper_chain[...,10] = np.degrees(flatchain[...,10]) # Omega_2
paper_chain[...,11:13] = flatchain[...,11:13] # M_A and M_B
paper_chain[...,13] = 1047.58*flatchain[...,13] # M_p
paper_chain[...,14:] = flatchain[...,14:] # k_1, gamma(s)

best_indx_paper = prob[:,5000:].flatten().argmax()

In [ ]:
table = pd.DataFrame(columns=['Best fit', 'Median', 'Upper uncertainty', 'Lower uncertainty', 'Uncertainty'], 
                     index=['P_1', 'T_01', 'e_1', 'i_1', '\omega_1', 'P_2', 'T_P2', 
                            'e_2', 'i_2', '\omega_2', '\Omega_2', 'M_A', 'M_B', 'm_p', 'k_2', '\gamma_1', '\gamma_2'])

table['Best fit'] = paper_chain[best_indx_paper]
table['Median'] = np.median(paper_chain, axis=0)
table['Upper uncertainty'] = np.abs(np.percentile(paper_chain, 84.15, axis=0) - np.percentile(paper_chain, 50, axis=0))
table['Lower uncertainty'] = np.abs(np.percentile(paper_chain, 15.85, axis=0) - np.percentile(paper_chain, 50, axis=0))
table['Uncertainty'] = paper_chain.std(axis=0)
table

# Misc

In [ ]:
plt.rc('font', size=14)
chain_carmenes = np.load('5095_chains_carmenes.npy')
prob_carmenes = np.load('5095_probs_carmenes.npy')
chain_not = np.load('5095_chains_not.npy')
prob_not = np.load('5095_probs_not.npy')
fig, axs = plt.subplots(2, 2, figsize=(10,9))
axs = axs.flatten()
def double_hist(ax, par):
    ax.hist(chain_carmenes[...,3000:,par].flatten(), bins=30, density=True, alpha=0.3, label='CARMENES')
    ax.hist(chain_not[...,3000:,par].flatten(), bins=30, density=True, alpha=0.3, label='NOT')
    ax.legend()
double_hist(axs[0], 11)
axs[0].set_xlabel('$M_A$ ($M_\odot$)')
double_hist(axs[1], 12)
axs[1].set_xlabel('$M_B$ ($M_\odot$)')
double_hist(axs[2], 3)
axs[2].set_xlabel('$e_1$')
double_hist(axs[3], 4)
axs[3].set_xlabel('$\omega_1$ (rad)')

In [ ]:
from etv import AnalyticETV
from scipy.stats import linregress

plt.figure(figsize=(14,8))
ts = np.linspace(0, 1600, 1000)
data = pd.read_csv('../data/KID5095269/koi509.tt.dan.db.try7.trans', 
                   header=None, delim_whitespace=True, index_col=0, names=['t', 'err'])
data.index = data.index.astype(int)
data.index = data.index + 41
p, t0, _, _, _ = linregress(data.index, data['t'])
data['etv'] = data['t'] - (p*data.index + t0)
plt.errorbar(data['t'], 86400*data['etv'], 86400*data['err'], linestyle='None', marker='x', c='k')

kic509 = AnalyticETV([18.61085, 66.86201, np.radians(81.35), 0.7057, np.radians(180+104.39),
                      235.8796, 95.923, 0.092*np.cos(np.radians(123)), 0.092*np.sin(np.radians(123)), np.radians(74.14), np.radians(99.4),
                      1.089, 1.037, 4.5/1047.5])
plt.plot(ts, 86400*kic509.del_tot_prim(ts))
